In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import tensorflow as tf
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)
#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
session_conf =tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

#tf.set_random_seed(seed)
tf.compat.v1.set_random_seed(seed)
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)

K.set_session(sess)
#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input, Flatten, Activation, Dropout, Layer
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras import optimizers,initializers,constraints,regularizers
from keras import backend as K
from keras.callbacks import LambdaCallback,ModelCheckpoint
from keras.utils import plot_model

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

import h5py
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pandas as pd
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split
import matplotlib.patches as mpatches
import matplotlib as mpl

#--------------------------------------------------------------------------------------------------------------------------------
#Import ourslef defined methods
import sys
sys.path.append(r"./Defined")
import Functions as F

# The following code should be added before the keras model
#np.random.seed(seed)

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
xy_train = np.load('bgedv2_XY_tr_float32.npy')
xy_validate = np.load('bgedv2_XY_va_float32.npy')

print('Shape of xy_train: ' + str(xy_train.shape)) 
print('Shape of xy_validate: ' + str(xy_validate.shape)) 

xy_test = np.load('bgedv2_XY_te_float32.npy')
print('Shape of xy_test: ' + str(xy_test.shape))

Shape of xy_train: (88807, 10463)
Shape of xy_validate: (11101, 10463)
Shape of xy_test: (11101, 10463)


In [3]:
xy_train_validate=np.r_[xy_train,xy_validate]
xy_train_validate.shape

(99908, 10463)

In [4]:
LM_ID = 'map_lm.txt'
TG_ID = 'map_tg_merge.txt'

LM_ID_infile = open(LM_ID)
LM_ID_infile_list=[]
for line in LM_ID_infile:
    LM_ID_infile_list.append(line.split('\t')[0])
    
TG_ID_infile = open(TG_ID)
TG_ID_infile_list=[]
for line in TG_ID_infile:
    TG_ID_infile_list.append(line.split('\t')[0])

In [5]:
LM_ID_arr=np.array(LM_ID_infile_list)
TG_ID_arr=np.array(TG_ID_infile_list)

In [6]:
LM_ID_arr

array(['UBE2C', 'RBKS', 'SLC2A6', 'DAXX', 'GNAS', 'H2AFV', 'ABHD6',
       'PGM1', 'CCNB1', 'S100A4', 'NOTCH1', 'TERT', 'BNIP3', 'ITGB1BP1',
       'ID2', 'POLR1C', 'TSPAN4', 'NSDHL', 'FAH', 'TRIM2', 'PAK6',
       'IKBKB', 'ACOT9', 'FBXO7', 'FYN', 'TRAP1', 'MBNL1', 'YTHDF1',
       'DFFB', 'BAX', 'HOOK2', 'ABCF3', 'SPRED2', 'ENOPH1', 'MYLK',
       'PRKACA', 'HPRT1', 'NUDT9', 'LSR', 'MALT1', 'PIN1', 'PPP2R5A',
       'ZMIZ1', 'CETN3', 'DUSP22', 'TRIB1', 'GTPBP8', 'LYPLA1', 'PIP4K2B',
       'ENOSF1', 'USP14', 'CDC25B', 'SCARB1', 'PYGL', 'CCNA2', 'IL13RA1',
       'PSME2', 'SPR', 'LRPAP1', 'PIK3C3', 'EML3', 'SERPINE1', 'GNAI1',
       'SMAD3', 'TGFBR2', 'PFKL', 'PLEKHJ1', 'ACAA1', 'RUVBL1', 'ABCC5',
       'DSG2', 'SLC25A13', 'SUPV3L1', 'MAPKAPK5', 'SCYL3', 'NFKBIB',
       'SLC25A46', 'GNAI2', 'DNAJC15', 'TRIB3', 'RFNG', 'TSEN2', 'KIF5C',
       'TXLNA', 'GDPD5', 'RAI14', 'EIF5', 'PSMF1', 'KIAA0907', 'CCNA1',
       'DFFA', 'PCBD1', 'CGRRF1', 'MAP4K4', 'TARBP1', 'IL4R', 'ICAM3',
     

In [7]:
landmark_index=np.array([np.where(TG_ID_arr==LM_ID_arr_i)[0][0] for LM_ID_arr_i in LM_ID_arr])

np.intersect1d(TG_ID_arr[landmark_index],LM_ID_arr).shape

(943,)

In [8]:
train_input=xy_train_validate[:,landmark_index]
train_output=xy_train_validate

test_input=xy_test[:,landmark_index]
test_output=xy_test

In [9]:
from sklearn.linear_model import LinearRegression

def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR

In [10]:
train_feature_tuple=(train_input,train_output)
test_feature_tuple=(test_input,test_output)

reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
print(reconstruction_loss)

0.26325423264377185
